# Result Arrangement

In [1]:
import pandas as pd
from glob import glob
import os
import pickle
import sys
import re
from collections import Counter

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
def CounterOutcome(row, categoryClass):
    from collections import Counter
    id_ = row.name
    target = row[['LogisticRegression',
       'NeuralNetwork_1', 'NeuralNetwork_2', 'RandomForestClassifier',
       'SVC', 'XGBoost']]
    counter = dict(Counter(target))
    notInCounter = list(filter(lambda x: x not in counter.keys(), categoryClass.classes_))
    others = dict(zip(notInCounter, [0] * len(notInCounter)))
    counter.update(others)
    counter['id'] = id_
    return counter

In [3]:
plt.rcParams["font.family"]

['NanumGothicOTF']

In [4]:
import Basic_Module as bm

## Category 

In [5]:
if os.path.isfile('./data/pre_data/news_tagged_data/pre_data_category_label_encoder_by_ct_for_doc2vec_news_classification.pickled'):
    le = pickle.load(open('./data/pre_data/news_tagged_data/pre_data_category_label_encoder_by_ct_for_doc2vec_news_classification.pickled','rb'))
else:
    le = LabelEncoder()
    le.fit(naverData['category'])
    pickle.dump(le, open('./data/pre_data/news_tagged_data/pre_data_category_label_encoder_by_ct_for_doc2vec_news_classification.pickled','wb'))
print (le.classes_)

['IT/과학' '경제' '사회' '생활/문화' '세계' '스포츠' '연예' '정치']


## Path

In [6]:
if sys.platform =='darwin':
    loadModelPath = '/Volumes/disk1/news_model/'
elif sys.platform =='win32':
    loadModelPath = 'd:/news_model/'
daumNewsPath = './data/pre_data/news_daum_news/'
classifierPath = './data/pre_data/news_classifier/'
outpath = './outcome_news_classification/'

In [7]:
outcomeFileList = glob(outpath+'outcome_news_classification'+'*')
print (len(outcomeFileList))

18


In [8]:
outdict = dict()
for outcomeFile in outcomeFileList:
    fileName = os.path.split(outcomeFile)[1]
    df = pd.read_csv(outcomeFile, index_col = None, encoding='utf-8')
    df.set_index(keys='id', inplace=True)
    out = df.apply(lambda x: CounterOutcome(x, le), axis = 1)
    out = pd.DataFrame.from_records(out, columns=['id','정치','경제','사회','생활/문화','세계','IT/과학','연예','스포츠'])
    out.set_index(keys='id', inplace = True)
    modelIs = bm.ExtractModelType(outcomeFile)
    outdict[modelIs] = out

In [9]:
outdf = pd.DataFrame()
for idx,key in enumerate(outdict):
    data = outdict[key]
    if outdf.shape == (0, 0):
        outdf = data.copy()
    else:
        outdf = outdf.add(data)

In [10]:
outdf.head()

,정치,경제,사회,생활/문화,세계,IT/과학,연예,스포츠
id,,,,,,,,
5a2a61bf588c13481c229d1e,0,1,25,21,60,1,0,0
5a2a61bf588c13481c229d1f,0,0,0,108,0,0,0,0
5a2a61bf588c13481c229d20,1,0,106,0,1,0,0,0
5a2a61bf588c13481c229d21,1,89,14,0,0,1,3,0
5a2a61bf588c13481c229d22,0,0,108,0,0,0,0,0


In [11]:
lstOutDf = df.loc[:,['title','extracted_keywords']]
lstOutDf = lstOutDf.merge(outdf, right_index=True, left_index=True)
l2 = lstOutDf.loc[:,['정치', '경제', '사회', '생활/문화', '세계',
       'IT/과학', '연예', '스포츠']]

In [12]:
lstOutDf.head()

,title,extracted_keywords,정치,경제,사회,생활/문화,세계,IT/과학,연예,스포츠
id,,,,,,,,,,
5a2a61bf588c13481c229d1e,"""밤이 무섭다""..비아그라 공장 연기에 남성들 부작용 호소","{'공장', '건강', '남성들', '지역', '부작용', '연기', '세보 효과'}",0,1,25,21,60,1,0,0
5a2a61bf588c13481c229d1f,식후 커피·늦은 양치질..점심식사 후 하면 안 좋은 습관 3가지,"{'치아', '건강', '입냄새', '커피', '점심 식사', '낮잠', '디스크'...",0,0,0,108,0,0,0,0
5a2a61bf588c13481c229d20,"'십년지기 생매장' 진짜 이유는..""'청부 통정' 알려질까 봐""","{'경찰', '성관계', '지인', '앙심', '진술', '철원', '아들', '주...",1,0,106,0,1,0,0,0
5a2a61bf588c13481c229d21,"신영자, 억 소리나는 갑질","{'신영자 이사장', '혐의', '롯데', '검찰', '유통업체', '징역', '매...",1,89,14,0,0,1,3,0
5a2a61bf588c13481c229d22,"""배신하지마"" 20대女 살인 피의자 유치장서 공범 남친에 쪽지","{'경찰', '혐의', '폭행', '쪽지', '유치장', '남자친구', '과자', ...",0,0,108,0,0,0,0,0


## classifier의 수

In [13]:
x = l2.sum(axis = 1).values
x2 = np.unique(x)
print (x.ndim)
print (x2)

1
[108]


In [14]:
print ('classifier의 수 : {} 개'.format(x2[0]))

classifier의 수 : 108 개


## 여러 classifier로 다음 뉴스를 네이버 category로 분류해보았을때, 모든 classifier가 하나의 cateogy로 분류한 비율

In [15]:
oneCategoryOutput = lstOutDf[l2.eq(x2[0]).sum(axis = 1) == 1]
oneCategoryOutput.head()

,title,extracted_keywords,정치,경제,사회,생활/문화,세계,IT/과학,연예,스포츠
id,,,,,,,,,,
5a2a61bf588c13481c229d1f,식후 커피·늦은 양치질..점심식사 후 하면 안 좋은 습관 3가지,"{'치아', '건강', '입냄새', '커피', '점심 식사', '낮잠', '디스크'...",0,0,0,108,0,0,0,0
5a2a61bf588c13481c229d22,"""배신하지마"" 20대女 살인 피의자 유치장서 공범 남친에 쪽지","{'경찰', '혐의', '폭행', '쪽지', '유치장', '남자친구', '과자', ...",0,0,108,0,0,0,0,0
5a2a61bf588c13481c229d40,"""사장님과 식사할 땐 턱받이까지 매드려야 해요""","{'장기자랑', '직원들', '사례', '119', '제보', '병원', '직장 갑질'}",0,0,108,0,0,0,0,0
5a2a61bf588c13481c229d58,"'마이웨이' 박해미 ""아픔까지 안아 준 9세 연하 남편..결혼식 못 올렸다""","{'마이웨이', '인생', '박해미', '배우', '열정', '남편 황민', '사랑'}",0,0,0,0,0,0,108,0
5a2a61bf588c13481c229d77,"'감빵생활' 박해수, 정수정 버렸다..이규형x정해인 반전 [종합]","{'준호', '박해수', '감빵 생활', '제혁', '지호', '야구', '슬기'}",0,0,0,0,0,0,108,0


In [16]:
oneProp = oneCategoryOutput.shape[0]/lstOutDf.shape[0]
print ('비율 : {0:.4f}, 약 {1:.1f}%'.format(oneProp, oneProp * 100))

비율 : 0.1108, 약 11.1%


## 여러 classifier로 다음 뉴스를 네이버 category로 분류해보았을때, 특정 category가 75%이상 나온 비율

In [29]:
x = l2.divide(x2[0]).multiply(100) >= 75

gt75 = lstOutDf[x.sum(axis = 1) == 1]
gt75.head()

,title,extracted_keywords,정치,경제,사회,생활/문화,세계,IT/과학,연예,스포츠
id,,,,,,,,,,
5a2a61bf588c13481c229d1f,식후 커피·늦은 양치질..점심식사 후 하면 안 좋은 습관 3가지,"{'치아', '건강', '입냄새', '커피', '점심 식사', '낮잠', '디스크'...",0,0,0,108,0,0,0,0
5a2a61bf588c13481c229d20,"'십년지기 생매장' 진짜 이유는..""'청부 통정' 알려질까 봐""","{'경찰', '성관계', '지인', '앙심', '진술', '철원', '아들', '주...",1,0,106,0,1,0,0,0
5a2a61bf588c13481c229d21,"신영자, 억 소리나는 갑질","{'신영자 이사장', '혐의', '롯데', '검찰', '유통업체', '징역', '매...",1,89,14,0,0,1,3,0
5a2a61bf588c13481c229d22,"""배신하지마"" 20대女 살인 피의자 유치장서 공범 남친에 쪽지","{'경찰', '혐의', '폭행', '쪽지', '유치장', '남자친구', '과자', ...",0,0,108,0,0,0,0,0
5a2a61bf588c13481c229d23,"화성서 발견된 포탄 모양 둥근 물체, 정체는?","{'공개', '모양 물체', '사진', 'NASA', '학자', '암석', '화성'}",0,0,1,2,3,102,0,0


In [19]:
prop2 = gt75.shape[0]/lstOutDf.shape[0]
print ('비율 : {0:.4f}, 약 {1:.1f}%'.format(prop2, prop2 * 100))

비율 : 0.7919, 약 79.2%


## 여러 classifier로 다음 뉴스를 네이버 category로 분류해보았을때, 특정 category가 50%보다 높게 나온 비율

In [32]:
x = l2.divide(x2[0]).multiply(100) > 50

gt50 = lstOutDf[x.sum(axis = 1) == 1]
gt50.head()

,title,extracted_keywords,정치,경제,사회,생활/문화,세계,IT/과학,연예,스포츠
id,,,,,,,,,,
5a2a61bf588c13481c229d1e,"""밤이 무섭다""..비아그라 공장 연기에 남성들 부작용 호소","{'공장', '건강', '남성들', '지역', '부작용', '연기', '세보 효과'}",0,1,25,21,60,1,0,0
5a2a61bf588c13481c229d1f,식후 커피·늦은 양치질..점심식사 후 하면 안 좋은 습관 3가지,"{'치아', '건강', '입냄새', '커피', '점심 식사', '낮잠', '디스크'...",0,0,0,108,0,0,0,0
5a2a61bf588c13481c229d20,"'십년지기 생매장' 진짜 이유는..""'청부 통정' 알려질까 봐""","{'경찰', '성관계', '지인', '앙심', '진술', '철원', '아들', '주...",1,0,106,0,1,0,0,0
5a2a61bf588c13481c229d21,"신영자, 억 소리나는 갑질","{'신영자 이사장', '혐의', '롯데', '검찰', '유통업체', '징역', '매...",1,89,14,0,0,1,3,0
5a2a61bf588c13481c229d22,"""배신하지마"" 20대女 살인 피의자 유치장서 공범 남친에 쪽지","{'경찰', '혐의', '폭행', '쪽지', '유치장', '남자친구', '과자', ...",0,0,108,0,0,0,0,0


In [33]:
prop3 = gt50.shape[0]/lstOutDf.shape[0]
print ('비율 : {0:.4f}, 약 {1:.1f}%'.format(prop3, prop3 * 100))

비율 : 0.9369, 약 93.7%


## 여러 classifier로 다음 뉴스를 네이버 category로 분류해보았을때, 특정 category가 50%인 뉴스의 비율

In [34]:
x = l2.divide(x2[0]).multiply(100) == 50

eq50 = lstOutDf[x.sum(axis = 1) == 1]
eq50.head()

,title,extracted_keywords,정치,경제,사회,생활/문화,세계,IT/과학,연예,스포츠
id,,,,,,,,,,
5a2aec8b588c132078bd207d,[뉴스+] 주말에 '집돌이·집순이' 되는 직장인들,"{'주말', '번아웃 증후군', '직장인', '스트레스', '증상', '교수', '...",0,2,54,48,3,1,0,0
5a2af415588c1376f0d67b16,'박근혜 5촌살인 의혹보도' 주진우·김어준 내일 대법원 선고,"{'사건', '연루', '혐의', '선고', '기자', '보도', '대통령', '조...",54,1,45,0,3,2,3,0
5a2b0aa2588c136a080754ec,"강남소방서 측 ""태연 가슴통증 호소..응급처치 특혜NO""","{'교통사고', '강남 소방서', '가슴 통증', '태연', '특혜'}",5,3,40,3,0,3,54,0
5a2b0eac588c13738825a5c2,"황우슬혜 ""꿀피부 비결은 아기 화장품, 비싼 것 필요 없어""","{'질문', '도전', '아기 화장품', '황우슬혜', '피부 관리', '역할'}",0,5,5,54,8,9,27,0
5a2c3b55588c135f3cb21ca8,"'이방인' 추신수 아내 하원미 ""삼남매 한방서 자는 것, 첫째 분리불안 때문""","{'김예은', '사진 이방인 캡처', '하원미', '경기', '삼남매', '이방인 ...",0,0,7,4,54,0,42,1


In [35]:
prop3 = eq50.shape[0]/lstOutDf.shape[0]
print ('비율 : {0:.4f}, 약 {1:.1f}%'.format(prop3, prop3 * 100))

비율 : 0.0053, 약 0.5%


## 여러 classifier로 다음 뉴스를 네이버 category로 분류해보았을때, 특정 category가 50%이상인 것이 하나도 없는 뉴스의 비율

In [36]:
x = l2.divide(x2[0]).multiply(100) >= 50

lt50 = lstOutDf[x.sum(axis = 1) == 0]
lt50.head()

,title,extracted_keywords,정치,경제,사회,생활/문화,세계,IT/과학,연예,스포츠
id,,,,,,,,,,
5a2a61bf588c13481c229d4f,"문 대통령, '에너지제로 주택 신혼부부 입주세대 방문'","{'주택', '신혼부부', '서울 노원구', '대통령', '오픈 하우스', '에너지...",46,36,0,8,3,2,13,0
5a2a61bf588c13481c229d50,"[인터뷰]MBC 신동진 아나운서 ""6년만의 방송, 울컥하면 안되는데..""","{'부당 전보', '파업', '방송', '아나운서', '부서', '사장', '다시'}",2,4,32,24,1,20,25,0
5a2a61bf588c13481c229d5a,"[단독] ""윤계상, 합의 제안했다"" 탈세 주장 시위자, '무고죄 맞고소' (전문)","{'신고', '침대 업체', '허위', '소송', '주장', '윤계상 탈세'}",1,20,12,3,1,19,52,0
5a2a61bf588c13481c229d62,"이창명 ""그날 사건, 술 아닌 하늘의 경고""(인터뷰②)","{'편집', '스타', '이창명', '녹화', '만원', '전진학', '출발 드림팀...",5,9,33,17,3,4,37,0
5a2a61bf588c13481c229d6f,"'라디오스타' 손동운 ""회사 지분 51%, 매출 1위는 윤두준""","{'하이라이트 손동운', '51 매출', '멤버', '회사'}",1,17,3,0,6,36,45,0


In [37]:
prop3 = lt50.shape[0]/lstOutDf.shape[0]
print ('비율 : {0:.4f}, 약 {1:.1f}%'.format(prop3, prop3 * 100))

비율 : 0.0577, 약 5.8%


## 결론

* Word2Vec, Doc2Vec, FastText로부터 만들어진 모델을 통해 단어 또는 문서를 vectorization을 하고 이를 통해 category를 분류해보았다. 
* 결과를 보면, 만들어진 classifier가 단일 결과를 내놓는 경우는 전체의 11.1%정도 였으며, 절반이상이 단일 결과를 내놓은 경우는 94.2%정도였다. 